In [2]:
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream

from preprocess import read_slang_words
from preprocess import handle_emojis
from preprocess import preprocess_word
from preprocess import is_valid_word

from pathlib import Path
import json
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import re


MAX_TWEETS = 100


ACCESS_TOKEN = '1213141429876248576-tebKwObkKvNxKTK0wHcGsC6us1yY0p'
ACCESS_SECRET = 'VPq8rWcmHCJ1MK7UcCCNUnNDg37wUlCCUAY4oD0hVK8gR'
CONSUMER_KEY = 'kDPOVCbOQBvleDpuWBMp7E8c3'
CONSUMER_SECRET = '7zkS43Vdjx52lIBxMXoldehhAmWTygFq3I01vtUNatUeOk6xJM'


CPT = 0
data_folder = Path("../dataset")
slang_words = "slang_words.csv"
slang_words_path = data_folder / slang_words
slang_words_dict = read_slang_words(str(slang_words_path))
stop_words_dict = set(stopwords.words('english'))
hash_tag_list = ["ISIS", "hillary clinton", "barack obama", "bernie sanders"]
fetched_tweets_filename = "tweets.csv"


use_stemmer = False
use_lemmatizer = True


after_pre_traitement = "tweets_after_pre_traitement.csv"




def preprocess_tweet(tweet):
   
    processed_tweet = []
    # Convert to lower case
    tweet = tweet.lower()
    # Replaces URLs with the word URL
    tweet = re.sub(r'((www\.[\S]+)|(https?://[\S]+))', ' URL ', tweet)
    # Replace @handle with the word USER_MENTION
    tweet = re.sub(r'@[\S]+', '', tweet)
    # Replaces #hashtag with hashtag
    tweet = re.sub(r'#(\S+)', r' \1 ', tweet)
    # Remove RT (retweet)
    tweet = re.sub(r'\brt\b', '', tweet)
    # Replace 2+ dots with space
    tweet = re.sub(r'\.{2,}', ' ', tweet)
    # Strip space, " and ' from tweet
    tweet = tweet.strip(' "\'')
    # Replace emojis with either EMO_POS or EMO_NEG
    tweet = handle_emojis(tweet)

    # Replace multiple spaces with a single space
    tweet = re.sub(r'\s+', ' ', tweet)

    words = tweet.split()
    for word in words:
        if word in slang_words_dict:
            tweet = re.sub(word.encode('raw-unicode-escape'),slang_words_dict.get(word)+" ", tweet)

    
    # Replace multiple spaces with a single space
    tweet = re.sub(r'\s+', ' ', tweet)

    # delete stop words 3ayatni         
    words = tweet.split() 
           
    for word in words:
        if word in stop_words_dict:
            tweet = tweet.replace(" "+word+" ", " ")
            
            

    #replace hmmm ta3 zabi
    tweet = re.sub(r'h(m)+','',tweet)
    

    # Replace multiple spaces with a single space
    tweet = re.sub(r'\s+', ' ', tweet)

    
    words = tweet.split()
    
    
   

    for word in words:
        word = preprocess_word(word)
        if is_valid_word(word):

            if use_lemmatizer and use_stemmer:
                word = str(porter_lemmatizer.lemmatize(word))
                word = str(porter_stemmer.stem(word))
            elif use_stemmer:
                word = str(porter_stemmer.stem(word))
            elif use_lemmatizer:
               word = str(porter_lemmatizer.lemmatize(word)) 

            processed_tweet.append(word)

    return ' '.join(processed_tweet)

# # # # TWITTER STREAMER # # # #
class TwitterStreamer():
    """
    Class for streaming and processing live tweets.
    """
    def __init__(self):
        pass

    def stream_tweets(self, fetched_tweets_filename,after_pre_traitement,hash_tag_list):
        # This handles Twitter authetification and the connection to Twitter Streaming API
        listener = StdOutListener(fetched_tweets_filename,after_pre_traitement)    
        auth = OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
        auth.set_access_token(ACCESS_TOKEN,ACCESS_SECRET)
        stream = Stream(auth, listener)
         # This line filter Twitter Streams to capture data by the keywords:
        stream.filter(track=hash_tag_list,languages=["en"])
        

        

        

# # # # TWITTER STREAM LISTENER # # # #
class StdOutListener(StreamListener):
    """
    This is a basic listener that just prints received tweets to stdout.
    """
    def __init__(self, fetched_tweets_filename,after_pre_traitement):
        self.fetched_tweets_filename = fetched_tweets_filename
        self.after_pre_traitement = after_pre_traitement

    def on_data(self, data):
        global CPT
        global MAX_TWEETS
        try:
            if CPT < MAX_TWEETS :
                CPT += 1
                all_data = json.loads(data)
            
                tweet_text = all_data['text'].encode('utf-8')
                ##print (tweet_text)
                tweet_id = all_data['id']
           
               
                processed_tweet = preprocess_tweet(tweet_text)
                with open(self.fetched_tweets_filename, 'a+') as tf:
                    tf.write('%s,%s\n' %(str(tweet_id), tweet_text))
                    
                    
                    with open(self.after_pre_traitement, 'a+') as tfclean:
                        if processed_tweet!="": 
                            tfclean.write('%s,%s\n' %(str(tweet_id),processed_tweet))
                        
                return True
            else:
                exit
        except BaseException as e:
            print("Error on_data %s" % str(e))
        return True
          

    def on_error(self, status):
        print(status)


if __name__ == '__main__':
 
   
    if use_lemmatizer and use_stemmer:
        porter_lemmatizer = WordNetLemmatizer() 
        porter_stemmer = PorterStemmer()
    elif use_lemmatizer :
        porter_lemmatizer = WordNetLemmatizer() 
    elif use_stemmer :
        porter_stemmer = PorterStemmer()
    
    
    twitter_streamer = TwitterStreamer()
    twitter_streamer.stream_tweets(fetched_tweets_filename,after_pre_traitement, hash_tag_list)
    

Processing 14/14